# INTDB Profile Mapping

    Query current INTDB profiles. Need rate classes & profiles to compare with ESG & Intellometry. Process enhancements needed for rate class parsing, filling in missing profiles, tags, etc. etc.

    How to best use ESG, Intellometry & EWX as datasources?

## Import Modules

    use requirements.txt instead
    set venv on server?

In [4]:
import numpy as np
import datetime as dt
import pandas as pd
import os
import time
import math
import json
import cx_Oracle
base = os.getcwd()

## DB Connection

In [2]:
''' connect to TPINT (or TPPE, LPSS) '''
''' parameters: {'query' : 'str of SQL query'} '''
''' output: {'result_list' : 'list of results returned from query'} '''

def OracleAPI(query):
    
    uid = 'tesi_interface'
    pwd = 'intint88'

    ip = '172.25.152.12'
    port = '1737'
    service_name = 'tpint.mytna.com'
    #dsn = cx_Oracle.makedsn(ip, port, service_name=service_name)
    
    print('querying...')
    
    con = cx_Oracle.connect(uid, pwd, service_name)
    cur = con.cursor()
    cur.execute(query)
    columns = [i[0] for i in cur.description]

    result_list = []
    
    for result in cur:
        result_list.append(result)
        i = len(result_list)
        if (i > 0 and i % 1000 == 0):
            print('done with {}.'.format(i))
    
    print('finished with {} results, outputting dataframe.'. format(len(result_list)))
    result = pd.DataFrame(result_list)
    result.columns = columns
    
    return(result)

## INTDB Query

In [6]:
INTDB = "select * from M_ESG_PC_RC_SOURCE a, m_lpss_pc_rc b where A.SOURCEID = B.ESG_SOURCEID"

In [5]:
other_intdb =  "select * from m_lpss_pc_rc"
other = OracleAPI(other_intdb)
other.head()

querying...
done with 1000.
done with 2000.
done with 3000.
done with 4000.
done with 5000.
done with 6000.
done with 7000.
finished with 7582 results, outputting dataframe.


,ESG_SOURCEID,MARKET,DISCO_CODE,PROFILE_CLASS,RATE_CLASS,METER_TYPE,LOSS_CODE,LOSS_DESCRIPTION,STATE,RES_FLAG,SEQ_ID,DEFAULT_FLAG
0,10001,ERCOT,CPT,None,ERCOT_BUS,IDR,D,SECONDARY,TX,0.0,1,N
1,10002,ERCOT,CPT,None,ERCOT_BUS,IDR,D,SECONDARY,TX,0.0,2,N
2,10003,ERCOT,CPT,None,ERCOT_BUS,IDR,E,PRIMARY,TX,0.0,3,N
3,10004,ERCOT,CPT,None,ERCOT_BUS,IDR,T,TRANSMISSION,TX,0.0,4,N
4,10005,ERCOT,CPT,None,ERCOT_BUS,IDR,D,SECONDARY,TX,0.0,5,Y


In [7]:
intdb_result = OracleAPI(INTDB)

querying...
done with 1000.
done with 2000.
done with 3000.
done with 4000.
done with 5000.
done with 6000.
done with 7000.
finished with 7582 results, outputting dataframe.


In [8]:
intdb_result.head()

,SOURCEID,MARKET,DISCO_CODE,LOAD_PROFILE,RATE_CLASS,ESG_SOURCEID,MARKET,DISCO_CODE,PROFILE_CLASS,RATE_CLASS,METER_TYPE,LOSS_CODE,LOSS_DESCRIPTION,STATE,RES_FLAG,SEQ_ID,DEFAULT_FLAG
0,10001,ERCOT,CPT,None,990,10001,ERCOT,CPT,None,ERCOT_BUS,IDR,D,SECONDARY,TX,0.0,1,N
1,10002,ERCOT,CPT,None,991,10002,ERCOT,CPT,None,ERCOT_BUS,IDR,D,SECONDARY,TX,0.0,2,N
2,10003,ERCOT,CPT,None,992,10003,ERCOT,CPT,None,ERCOT_BUS,IDR,E,PRIMARY,TX,0.0,3,N
3,10004,ERCOT,CPT,None,993,10004,ERCOT,CPT,None,ERCOT_BUS,IDR,T,TRANSMISSION,TX,0.0,4,N
4,10005,ERCOT,CPT,None,994,10005,ERCOT,CPT,None,ERCOT_BUS,IDR,D,SECONDARY,TX,0.0,5,Y


In [7]:
intdb_result.tail()

,SOURCEID,MARKET,DISCO_CODE,LOAD_PROFILE,RATE_CLASS,ESG_SOURCEID,MARKET,DISCO_CODE,PROFILE_CLASS,RATE_CLASS,METER_TYPE,LOSS_CODE,LOSS_DESCRIPTION,STATE,RES_FLAG,SEQ_ID,DEFAULT_FLAG
7577,54549,PJM,APSPA,GSCS,WP-GP30SHD,54549,PJM,APSPA,GSCS_IDR,WP-GP30SHD,IDR,PRIMARY,WP_PRI,PA,0.0,9599,N
7578,54550,PJM,APSPA,GSCS,WP-GP30SHD,54550,PJM,APSPA,GSCS_IDR,WP-GP30SHD,IDR,SUBTRANSMISSION,WP_SUB,PA,0.0,9600,Y
7579,54551,PJM,APSPA,GSCS,WP-GP30SHF,54551,PJM,APSPA,GSCS,WP-GP30SHF,SCALAR,SECONDARY,WP_SEC,PA,0.0,9601,Y
7580,54552,PJM,APSPA,GSCS,WP-GP30SHF,54552,PJM,APSPA,GSCS_IDR,WP-GP30SHF,IDR,SECONDARY,WP_SEC,PA,0.0,9602,Y
7581,54553,PJM,PEPCODC,RDNS,150,54553,PJM,PEPCODC,RDNS,50,IDR,MMA1,None,DC,0.0,9603,Y


## INTDB Results

    Want to find missing profiles, differences between ESG and INTDB, places with large # parsing errors.

In [8]:
intdb = intdb_result[['SOURCEID', 'MARKET', 'DISCO_CODE', 'LOAD_PROFILE', 'RATE_CLASS']]
intdb.head()

,SOURCEID,MARKET,MARKET,DISCO_CODE,DISCO_CODE,LOAD_PROFILE,RATE_CLASS,RATE_CLASS
0,10001,ERCOT,ERCOT,CPT,CPT,None,990,ERCOT_BUS
1,10002,ERCOT,ERCOT,CPT,CPT,None,991,ERCOT_BUS
2,10003,ERCOT,ERCOT,CPT,CPT,None,992,ERCOT_BUS
3,10004,ERCOT,ERCOT,CPT,CPT,None,993,ERCOT_BUS
4,10005,ERCOT,ERCOT,CPT,CPT,None,994,ERCOT_BUS


In [9]:
intdb.shape

(7582, 8)

# ALPS Missing Mappings

In [10]:
alps = pd.read_csv('ALPS_PARSER_MISSING_MAPS.csv')
alps.head()

,UID_MAP,CREATED_USER,CREATED_DT,UPDATED_USER,UPDATED_DT,MARKET_CODE,DISCO_CODE,ZONE,RATE,PROFILE,...,METER_TYPE,STRATA,OUT_ZONE,OUT_RATE,OUT_PROFILE,OUT_VOLTAGE,OUT_METER_TYPE,OUT_STRATA,OUT_IGNORE_FLG,COMMENTS
0,"1,130,355",TESI_INTERFACE,08/29/2016 11:43:44,TESI_INTERFACE,06/06/2017 10:35:10,NEPOOL,COMELEC,SEMA,R-5,NaN,...,NaN,NaN,SEMA,R5,R1,SECONDARY,NaN,NaN,N,SERNA_USGH_20170601_133012_R143490112_USGH_USG...
1,"1,131,086",TESI_INTERFACE,08/30/2016 07:06:20,TESI_INTERFACE,06/04/2017 12:36:55,PJM,ATSITE,FEOHIOTE,TE-POLSD,SL,...,NaN,NaN,FEOHIOTE,TE-POLSD,SL,NaN,NaN,NaN,N,SERNA_USGH_20170604_131356_R143638625_USGH_USG...
2,"1,131,109",TESI_INTERFACE,08/30/2016 07:14:19,TESI_INTERFACE,04/20/2017 07:31:10,PJM,ATSITE,FEOHIOTE,TE-GTD,C1,...,NaN,NaN,FEOHIOTE,TE-GTD,C1,NaN,NaN,NaN,N,SERNA_USGH_20170420_074801_R140919998_USGH_USG...
3,"1,131,300",TESI_INTERFACE,08/30/2016 08:14:13,TESI_INTERFACE,03/15/2017 08:26:51,PJM,DPL,DPL,USGSSPHTG,MDDGS,...,NaN,NaN,DPL,USGSSPHTG,MDDGS,SECONDARY_120_480V,NaN,NaN,N,SERNA_USGH_20170314_091525_R138696040_USGH_USG...
4,"1,131,301",TESI_INTERFACE,08/30/2016 08:14:13,TESI_INTERFACE,06/02/2017 10:04:48,PJM,PEPCODC,PEPCODC,180,DCND,...,NaN,NaN,PEPCODC,80,DCND,GSLV,NaN,NaN,N,SERNA_USGH_20170602_101419_R143559487_USGH_USG...


In [11]:
alps

,UID_MAP,CREATED_USER,CREATED_DT,UPDATED_USER,UPDATED_DT,MARKET_CODE,DISCO_CODE,ZONE,RATE,PROFILE,...,METER_TYPE,STRATA,OUT_ZONE,OUT_RATE,OUT_PROFILE,OUT_VOLTAGE,OUT_METER_TYPE,OUT_STRATA,OUT_IGNORE_FLG,COMMENTS
0,"1,130,355",TESI_INTERFACE,08/29/2016 11:43:44,TESI_INTERFACE,06/06/2017 10:35:10,NEPOOL,COMELEC,SEMA,R-5,NaN,...,NaN,NaN,SEMA,R5,R1,SECONDARY,NaN,NaN,N,SERNA_USGH_20170601_133012_R143490112_USGH_USG...
1,"1,131,086",TESI_INTERFACE,08/30/2016 07:06:20,TESI_INTERFACE,06/04/2017 12:36:55,PJM,ATSITE,FEOHIOTE,TE-POLSD,SL,...,NaN,NaN,FEOHIOTE,TE-POLSD,SL,NaN,NaN,NaN,N,SERNA_USGH_20170604_131356_R143638625_USGH_USG...
2,"1,131,109",TESI_INTERFACE,08/30/2016 07:14:19,TESI_INTERFACE,04/20/2017 07:31:10,PJM,ATSITE,FEOHIOTE,TE-GTD,C1,...,NaN,NaN,FEOHIOTE,TE-GTD,C1,NaN,NaN,NaN,N,SERNA_USGH_20170420_074801_R140919998_USGH_USG...
3,"1,131,300",TESI_INTERFACE,08/30/2016 08:14:13,TESI_INTERFACE,03/15/2017 08:26:51,PJM,DPL,DPL,USGSSPHTG,MDDGS,...,NaN,NaN,DPL,USGSSPHTG,MDDGS,SECONDARY_120_480V,NaN,NaN,N,SERNA_USGH_20170314_091525_R138696040_USGH_USG...
4,"1,131,301",TESI_INTERFACE,08/30/2016 08:14:13,TESI_INTERFACE,06/02/2017 10:04:48,PJM,PEPCODC,PEPCODC,180,DCND,...,NaN,NaN,PEPCODC,80,DCND,GSLV,NaN,NaN,N,SERNA_USGH_20170602_101419_R143559487_USGH_USG...
5,"1,131,302",TESI_INTERFACE,08/30/2016 08:14:13,TESI_INTERFACE,04/07/2017 09:04:11,PJM,DPL,DPL,DMEL_OL,MDDOL,...,NaN,NaN,DPL,DMEL_OL,MDDOL,NaN,NaN,NaN,N,SERNA_USGH_20170407_093510_R140167172_USHI_USG...
6,"1,131,303",TESI_INTERFACE,08/30/2016 08:14:13,TESI_INTERFACE,03/04/2017 08:46:22,PJM,DPL,DPL,UGSPTOU,MDDGP,...,NaN,NaN,DPL,UGSPTOU,MDDGP,NaN,NaN,NaN,N,SERNA_USGH_20170304_090014_R138147247_USHI_USG...
7,"1,131,353",TESI_INTERFACE,08/30/2016 08:59:42,TESI_INTERFACE,05/22/2017 16:14:21,PJM,AEPOP,AEPOP,900,CCOPSLTA,...,NaN,NaN,AEPOP,900,CCOPSLTA,NaN,NaN,NaN,N,SERNA_USGH_20170522_165539_R142882940_USGH_USG...
8,"1,131,354",TESI_INTERFACE,08/30/2016 08:59:42,TESI_INTERFACE,05/22/2017 16:14:21,PJM,AEPOP,AEPOP,901,CCOPSLTA,...,NaN,NaN,AEPOP,901,CCOPSLTA,NaN,NaN,NaN,N,SERNA_USGH_20170522_165539_R142882940_USGH_USG...
9,"1,131,449",TESI_INTERFACE,08/30/2016 09:43:47,TESI_INTERFACE,03/18/2017 10:50:00,MISO,AMERENIP,AMILSZE,DS5,LITE,...,NaN,NaN,AMILSZE,DS5,LITE,NaN,NaN,NaN,N,SERNA_USGH_20170318_102438_R138978619_USHI_USG...


In [12]:
alps.shape

(1513, 21)

# Compare ALPS vs INTDB

    group data by (market, disco)

In [9]:
other['GROUP'] = list(zip(other['MARKET'], other['DISCO_CODE'], other['RATE_CLASS']))

In [14]:
alps['GROUP'] = list(zip(alps['MARKET_CODE'], alps['DISCO_CODE'], alps['RATE']))

In [10]:
other['GROUP'].head()

0    (ERCOT, CPT, ERCOT_BUS)
1    (ERCOT, CPT, ERCOT_BUS)
2    (ERCOT, CPT, ERCOT_BUS)
3    (ERCOT, CPT, ERCOT_BUS)
4    (ERCOT, CPT, ERCOT_BUS)
Name: GROUP, dtype: object

In [32]:
intdb_result['INTEL_GROUP'] = list(zip(intdb_result['DISCO_CODE'], intdb_result['LOAD_PROFILE'], intdb_result['RATE_CLASS']))

ValueError: Length of values does not match length of index

In [16]:
common = pd.merge(other, alps['GROUP'], how = 'inner', on = 'GROUP')
common.drop_duplicates(keep = 'first',inplace = True)
common.reset_index(drop = True, inplace = True)

## Find common (market, disco, rate)

In [17]:
common

,ESG_SOURCEID,MARKET,DISCO_CODE,PROFILE_CLASS,RATE_CLASS,METER_TYPE,LOSS_CODE,LOSS_DESCRIPTION,STATE,RES_FLAG,SEQ_ID,DEFAULT_FLAG,GROUP
0,10001,ERCOT,CPT,None,ERCOT_BUS,IDR,D,SECONDARY,TX,0.0,1,N,"(ERCOT, CPT, ERCOT_BUS)"
1,10002,ERCOT,CPT,None,ERCOT_BUS,IDR,D,SECONDARY,TX,0.0,2,N,"(ERCOT, CPT, ERCOT_BUS)"
2,10003,ERCOT,CPT,None,ERCOT_BUS,IDR,E,PRIMARY,TX,0.0,3,N,"(ERCOT, CPT, ERCOT_BUS)"
3,10004,ERCOT,CPT,None,ERCOT_BUS,IDR,T,TRANSMISSION,TX,0.0,4,N,"(ERCOT, CPT, ERCOT_BUS)"
4,10005,ERCOT,CPT,None,ERCOT_BUS,IDR,D,SECONDARY,TX,0.0,5,Y,"(ERCOT, CPT, ERCOT_BUS)"
5,10006,ERCOT,CPT,None,ERCOT_BUS,IDR,D,SECONDARY,TX,0.0,6,Y,"(ERCOT, CPT, ERCOT_BUS)"
6,10007,ERCOT,CPT,None,ERCOT_BUS,IDR,E,PRIMARY,TX,0.0,7,Y,"(ERCOT, CPT, ERCOT_BUS)"
7,10008,ERCOT,CPT,None,ERCOT_BUS,IDR,T,TRANSMISSION,TX,0.0,8,Y,"(ERCOT, CPT, ERCOT_BUS)"
8,10009,ERCOT,CPT,None,ERCOT_BUS,IDR,D,SECONDARY,TX,0.0,9,N,"(ERCOT, CPT, ERCOT_BUS)"
9,10010,ERCOT,CPT,None,ERCOT_BUS,IDR,D,SECONDARY,TX,0.0,10,N,"(ERCOT, CPT, ERCOT_BUS)"


## Find Uncommon (not in ESG and not in INTDB)

In [21]:
other_group = list(set(alps.GROUP).symmetric_difference(set(common.GROUP)))
decide = [a in other_group for a in alps.GROUP]

other_group = alps[decide]
other_group.reset_index(drop = True, inplace = True)
other_group

,UID_MAP,CREATED_USER,CREATED_DT,UPDATED_USER,UPDATED_DT,MARKET_CODE,DISCO_CODE,ZONE,RATE,PROFILE,...,STRATA,OUT_ZONE,OUT_RATE,OUT_PROFILE,OUT_VOLTAGE,OUT_METER_TYPE,OUT_STRATA,OUT_IGNORE_FLG,COMMENTS,GROUP
0,"1,130,355",TESI_INTERFACE,08/29/2016 11:43:44,TESI_INTERFACE,06/06/2017 10:35:10,NEPOOL,COMELEC,SEMA,R-5,NaN,...,NaN,SEMA,R5,R1,SECONDARY,NaN,NaN,N,SERNA_USGH_20170601_133012_R143490112_USGH_USG...,"(NEPOOL, COMELEC, R-5)"
1,"1,131,300",TESI_INTERFACE,08/30/2016 08:14:13,TESI_INTERFACE,03/15/2017 08:26:51,PJM,DPL,DPL,USGSSPHTG,MDDGS,...,NaN,DPL,USGSSPHTG,MDDGS,SECONDARY_120_480V,NaN,NaN,N,SERNA_USGH_20170314_091525_R138696040_USGH_USG...,"(PJM, DPL, USGSSPHTG)"
2,"1,131,301",TESI_INTERFACE,08/30/2016 08:14:13,TESI_INTERFACE,06/02/2017 10:04:48,PJM,PEPCODC,PEPCODC,180,DCND,...,NaN,PEPCODC,80,DCND,GSLV,NaN,NaN,N,SERNA_USGH_20170602_101419_R143559487_USGH_USG...,"(PJM, PEPCODC, 180)"
3,"1,131,303",TESI_INTERFACE,08/30/2016 08:14:13,TESI_INTERFACE,03/04/2017 08:46:22,PJM,DPL,DPL,UGSPTOU,MDDGP,...,NaN,DPL,UGSPTOU,MDDGP,NaN,NaN,NaN,N,SERNA_USGH_20170304_090014_R138147247_USHI_USG...,"(PJM, DPL, UGSPTOU)"
4,"1,131,790",TESI_INTERFACE,08/30/2016 13:44:35,TESI_INTERFACE,10/24/2018 17:27:56,NEPOOL,CLP,CONNECTICUT,030,NaN,...,NaN,CONNECTICUT,030,NaN,NaN,NaN,NaN,N,SERNA_USGH_20180601_141150_R165565733_USGH_USG...,"(NEPOOL, CLP, 030)"
5,"1,131,881",TESI_INTERFACE,08/30/2016 14:41:39,TESI_INTERFACE,06/01/2017 13:50:40,PJM,BGE,NaN,INTERVAL GL,165,...,NaN,BGE,INTERVAL GL,165,NaN,NaN,NaN,N,SCA_BGE_0167102624_1-FDTX0P_1_20170601_120411_...,"(PJM, BGE, INTERVAL GL)"
6,"1,132,350",TESI_INTERFACE,08/30/2016 16:44:41,TESI_INTERFACE,05/08/2017 13:38:43,NYISO,RGE,GEN,RED0100E00,NaN,...,NaN,B,RED0100E00,NaN,SE,NaN,NaN,N,SCA_RGE_14_1-FB0K04_1_05_08_2017_13_30_54.txt,"(NYISO, RGE, RED0100E00)"
7,"1,132,351",TESI_INTERFACE,08/30/2016 16:44:41,TESI_INTERFACE,05/08/2017 13:38:43,NYISO,RGE,GEN,RED0420E00,NaN,...,NaN,B,RED0420E00,NaN,SE,NaN,NaN,N,SCA_RGE_14_1-FB0K04_1_05_08_2017_13_30_54.txt,"(NYISO, RGE, RED0420E00)"
8,"1,132,352",TESI_INTERFACE,08/30/2016 16:44:41,TESI_INTERFACE,05/04/2017 17:03:53,NYISO,RGE,GEN,RED0100V00,NaN,...,NaN,B,RED0100V00,NaN,SE,NaN,NaN,N,SCA_RGE_27_1-FAF3GC_1_05_04_2017_16_51_03.txt,"(NYISO, RGE, RED0100V00)"
9,"1,134,498",TESI_INTERFACE,09/01/2016 11:41:56,TESI_INTERFACE,09/01/2016 11:41:56,ERCOT,ONCOR,NaN,R8,NMLIGHT_EAST_NIDR_NWS_NOTOU,...,NaN,NaN,ERCOT_NM,NMLIGHT_EAST,A,NaN,NaN,N,B6S7MV9_1_FUMC UsageData.xlsx,"(ERCOT, ONCOR, R8)"


In [29]:
uniq_other_group = other_group.drop_duplicates(['MARKET_CODE', 'DISCO_CODE', 'RATE'], keep = 'first')
uniq_other_group.reset_index(drop = True, inplace = True)

In [37]:
uniq_common = common.drop_duplicates(['MARKET', 'DISCO_CODE', 'RATE_CLASS', 'PROFILE_CLASS', 'METER_TYPE', 'LOSS_CODE', 'LOSS_DESCRIPTION'], keep = 'first')
uniq_common.reset_index(drop = True, inplace = True)

# Intelometry

In [26]:
intel = pd.read_csv('Utility_Rate_Code_Mapping.csv')
intel['Profile'] = intel['Profile'].astype('str')
intel.head()

,Utility Name,Rate Name,Description,Commodity,Voltage,Business Type,Loss Factor,Profile
0,AECO,AECO_RC_AGSPRIMARY,Annual General Service Prmary,Power,NaN,General Service,NaN,AECO_LP_NJAAP
1,AECO,AECO_RC_AGSPRIMARYH,"Annual General Service, Primary",Power,NaN,General Service,NaN,AECO_LP_NJAAP
2,AECO,AECO_RC_AGSSECOND,"Annual General Service, Secondary",Power,Secondary,General Service,NaN,AECO_LP_NJAAS
3,AECO,AECO_RC_AGSSECONDH,"Annual General Service, Secondary",Power,Secondary,General Service,NaN,AECO_LP_NJAAS
4,AECO,AECO_RC_ANEL_CSL,Street and Private Lighting,Power,Secondary,Ltg-St,NaN,AECO_LP_NJAOL


In [29]:
rate = [r.split('_')[2:] for r in intel['Rate Name']]
intel['Rate'] = ['_'.join(r) for r in rate]
intel['Load'] = [p.split('_') for p in intel['Profile']]
load = []
for p in intel['Load']:
    try:
        load.append(p[2])
    except:
        load.append('NA')
intel['Load'] = load
intel.head()

,Utility Name,Rate Name,Description,Commodity,Voltage,Business Type,Loss Factor,Profile,Rate,Load
0,AECO,AECO_RC_AGSPRIMARY,Annual General Service Prmary,Power,NaN,General Service,NaN,AECO_LP_NJAAP,AGSPRIMARY,NJAAP
1,AECO,AECO_RC_AGSPRIMARYH,"Annual General Service, Primary",Power,NaN,General Service,NaN,AECO_LP_NJAAP,AGSPRIMARYH,NJAAP
2,AECO,AECO_RC_AGSSECOND,"Annual General Service, Secondary",Power,Secondary,General Service,NaN,AECO_LP_NJAAS,AGSSECOND,NJAAS
3,AECO,AECO_RC_AGSSECONDH,"Annual General Service, Secondary",Power,Secondary,General Service,NaN,AECO_LP_NJAAS,AGSSECONDH,NJAAS
4,AECO,AECO_RC_ANEL_CSL,Street and Private Lighting,Power,Secondary,Ltg-St,NaN,AECO_LP_NJAOL,ANEL_CSL,NJAOL


In [22]:
intel.tail()

,Utility Name,Rate Name,Description,Commodity,Voltage,Business Type,Loss Factor,Profile,Rate
3250,WMECO,WMECO_RC_S2,"Street Lighting (S1, S2) Customer Owned Facili...",Power,Secondary,Ltg-St,WMECO_LF_SEC,WMECO_LP_S1,S2
3251,WMECO,WMECO_RC_T0,Small General Service - Time-of-Use,Power,Secondary,GenSvc-Sm,WMECO_LF_SEC,WMECO_LP_T2,T0
3252,WMECO,WMECO_RC_T2,Large TOU Primary Service (T2),Power,Primary,GenSvc-Lg-TOU,WMECO_LF_PRI,WMECO_LP_T2,T2
3253,WMECO,WMECO_RC_T4,"Intermediate Primary General Service (G2, T4)",Power,Primary,GenSvc-Med,WMECO_LF_PRI,WMECO_LP_G2,T4
3254,WMECO,WMECO_RC_T5,"Extra Large Primary Service, Time-of-Use (T5)",Power,Primary,GenSvc-Lg-TOU,WMECO_LF_PRI,NaN,T5


In [ ]:
intel_group = 

## Write to csv

In [19]:
common.to_csv('INTDB_ALPS_MISSING_OVERLAP.csv')

In [20]:
other_group.to_csv('INDTB_ALPS_MISSING_BOTH.csv')

In [40]:
uniq_common.to_csv('INTDB_ALPS_MISSING_OVERLAP_UNIQ.csv')

'C:\\Users\\wb5888\\LA-tools\\package'